## Point Registration

This tutorial shows you how to register 'nii' files with points. The points can be computed by segmentations like in Verse19.

You can get the Verse19 data from:

https://osf.io/nqjyw/

We assume in this tutorial that you copied the "dataset-verse19validation" folder next to this file.


In [1]:
from pathlib import Path
from TPTBox.registration.ridged_points import ridged_points_from_subreg_vert
from TPTBox import NII, POI, calc_centroids_from_subreg_vert
from TPTBox.snapshot2D import Snapshot_Frame, create_snapshot
from IPython.display import Image
from TPTBox.core.nii_wrapper import to_nii

loading data

In [2]:
# TODO update paths
ct1 = Path(
    "/media/data/robert/datasets/dataset-neuropoly/rawdata_upscale/sub-m002126/ses-20100716/anat/sub-m002126_ses-20100716_acq-sag_desc-stitched_T2w.nii.gz"
).absolute()
ct2 = Path(
    "/media/data/robert/datasets/dataset-neuropoly/rawdata/sub-m002126/ses-20100716/anat/sub-m002126_ses-20100716_acq-ax_chunk-2_T2w.nii.gz"
).absolute()


def to_poi(p: Path):
    return Path(str(p).replace("rawdata", "derivatives").replace("_seg-vert_msk.nii.gz", "_seg-spine_ctd.json"))


def to_subreg(p: Path):
    return Path(str(p).replace("rawdata", "derivatives").replace("_seg-vert_msk.nii.gz", "_seg-spine_msk.nii.gz"))


c1_vert = Path(
    "/media/data/robert/datasets/dataset-neuropoly/derivatives_segnew/sub-m002126/ses-20100716/anat/sub-m002126_ses-20100716_sequ-stitched_acq-sag_mod-T2w_seg-vert_msk.nii.gz"
)

c2_vert = Path(
    "/media/data/robert/datasets/dataset-neuropoly/derivatives_ax/sub-m002126/ses-20100716/anat/sub-m002126_ses-20100716_acq-ax_chunk-2_mod-T2w_seg-vert_msk.nii.gz"
)

c1_poi = to_poi(c1_vert)
c2_poi = to_poi(c2_vert)
c1_subreg = to_subreg(c1_vert)
c2_subreg = to_subreg(c2_vert)
print(ct1.exists(), ct1)
print(c1_poi.exists(), c2_poi.exists())
print(c1_poi)

False /media/data/robert/datasets/dataset-neuropoly/rawdata_upscale/sub-m002126/ses-20100716/anat/sub-m002126_ses-20100716_acq-sag_desc-stitched_T2w.nii.gz
False False
/media/data/robert/datasets/dataset-neuropoly/derivatives_segnew/sub-m002126/ses-20100716/anat/sub-m002126_ses-20100716_sequ-stitched_acq-sag_mod-T2w_seg-spine_ctd.json


### From Centroid or Points of Interest(POI) list

We load the POIs form the json. It should have the following shape:
```[
    {
        "direction": ["P","I","R"]
    },
    {
        "label": 20, "X": 91.1,"Y": 40.0, "Z": 95.0
    },
    {
        "label": 21, "X": 76.5,"Y": 68.1, "Z": 90.3
    },
    {
        "label": 22, "X": 63.6,"Y": 101.2, "Z": 86.3
    },
    {
        "label": 23, "X": 59.0,"Y": 136.8, "Z": 86.8
    },
    {
        "label": 24, "X": 72.1,"Y": 166.6, "Z": 94.0
    }
]
```

where the coordinate are the local cords of the image

In [ ]:
ct1_nii_org = to_nii(ct1, False)  # .resample_from_to(c1_vert)
ct2_nii = to_nii(ct2, False).resample_from_to(c2_vert)
c1_vert_nii_org = to_nii(c1_vert, True)

In [ ]:
from TPTBox.core.nii_wrapper import to_nii

orientation = ("L", "A", "S")
zoom = (0.8571, 0.8571, 0.8571)
c2_poi_ = calc_centroids_from_subreg_vert(c2_vert, c2_subreg, subreg_id=[50, 61])
resample_filter = ridged_points_from_subreg_vert(
    c2_poi_, c1_vert, c1_subreg, c1_poi, orientation=orientation, zoom=zoom, subreg_id=[50, 61], c_val=0, verbose=False
)
rep_moving_nii: NII = resample_filter.transform_nii(to_nii(ct2, False))
rep_seg_moving_nii: NII = resample_filter.transform_nii(to_nii(c2_vert, True))
rep_seg2_moving_nii: NII = resample_filter.transform_nii(to_nii(c2_subreg, True))
poi_new = resample_filter.transform_poi(c2_poi_)

c1_vert_iso = to_nii(c1_vert, True).rescale(zoom)
p1 = POI.load(c1_poi).rescale(zoom).extract_subregion_(50, 61)
c1_iso = to_nii(ct1, False).rescale(zoom)
# calc crop
# crop = rep_moving_nii.compute_crop()
# crop = c1_iso.compute_crop(other_crop=crop)

ct1_frame = Snapshot_Frame(c1_iso, segmentation=c1_vert_iso, centroids=p1)
ct2_frame = Snapshot_Frame(rep_moving_nii, segmentation=rep_seg_moving_nii, centroids=p1)
ct3_frame = Snapshot_Frame(to_nii(ct2, False).resample_from_to(rep_seg_moving_nii), segmentation=rep_seg_moving_nii, centroids=poi_new)
ct_org_frame = Snapshot_Frame(
    to_nii(ct2, False).resample_from_to(c1_iso), segmentation=c1_vert_iso, centroids=c2_poi_.resample_from_to(c1_iso)
)
create_snapshot("test.jpg", [ct1_frame, ct2_frame, ct3_frame, ct_org_frame])
Image(filename="test.jpg")

# Cropping
The moved images has some space unoccupied. We can remove non corresponding spaces by cropping

In [ ]:
from TPTBox import to_nii, NII  # noqa: I001

### OUTDATED ###

# orientation = rep_moving_nii.orientation
# zoom = rep_moving_nii.zoom


# NII is a wrapper around nibabel
ct1_nii = to_nii(ct1).reorient_(orientation).rescale_(zoom)
ct2_nii = rep_moving_nii.copy()
ct1_seg = to_nii(c1_vert, seg=True).reorient_(orientation).rescale_(zoom)
ct2_seg = rep_seg_moving_nii.copy()
assert ct1_nii.shape == ct2_nii.shape, (ct1_nii, ct2_nii)
ex_slice_f = ct1_nii.compute_crop()
shared_pixels = ct2_nii.compute_crop(other_crop=ex_slice_f)


c1_poi_cropped = POI.load(c1_poi).reorient_(orientation).rescale_(zoom).apply_crop(shared_pixels)
ct1_nii.apply_crop_(shared_pixels)
ct2_nii.apply_crop_(shared_pixels)
ct1_seg.apply_crop_(shared_pixels)
ct2_seg.apply_crop_(shared_pixels)

ct1_frame = Snapshot_Frame(ct1_nii, segmentation=ct1_seg, centroids=c1_poi_cropped)
ct2_frame = Snapshot_Frame(ct2_nii, segmentation=ct2_seg, centroids=c1_poi_cropped)
create_snapshot("test.jpg", [ct1_frame, ct2_frame])
Image(filename="test.jpg")

## Computing Centroids from segmentation

We can generate Centroids/POI by computing the Center Of Mass of Segmentation.

Variant 1: Just using the segmentation

In [ ]:
from TPTBox import calc_centroids

ctd = calc_centroids(c1_vert, decimals=0)
print(to_nii(c1_vert))
print("orientation", ctd.orientation)
print("shape", ctd.shape)
print("zoom", ctd.zoom)
print("points", ctd.keys())
print("id 9", ctd[9, 50])
print("\n", ctd)

Variant 2: Segmentation and Subregion Segmentation

If you have two files, where the second splits the other in smaller chunks you can use this variant. (Verse does not provide this...)

In [ ]:
from TPTBox import calc_centroids_from_subreg_vert  # noqa: I001


instance_nii = to_nii(c1_vert, seg=True)
arr = instance_nii.get_array()
arr[arr != 0] = 50
subregion_nii = instance_nii.set_array(arr, inplace=False)

ctd = calc_centroids_from_subreg_vert(instance_nii, subregion_nii, subreg_id=[50])
print(ctd)
print(list(ctd.items()))
print("Note: the ids are subregion_ID*256 + segmentation id. Matching points MUST have the same ID")